## Exercício: Modelo de Linguagem com auto-atenção e máscaras causais com ajuste fino usando LoRA

**Proposta**

Este projeto visa realizar um pré-treino de um modelo decoder em português, com o objetivo de gerar textos no estilo de Machado de Assis. O modelo será refinado em duas etapas: uma com todos os pesos atualizados e outra utilizando a técnica LoRA, na qual apenas as matrizes A e B das camadas lineares e embeddings são ajustadas.

**Método**

- **Dados**: Conjunto de contos de Machado de Assis.
- **Modelo**: Transformer com multi-head attention e máscara causal. Durante o refinamento, a técnica LoRA será aplicada em uma das versões, com atualização apenas das matrizes A e B, enquanto a outra versão atualizará todos os pesos.

## Dados

Vamos usar o mesmo dataset do Machado de Assis.



In [1]:
import os
import re
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from torch.nn import functional as F
from torch.nn import Embedding
from collections import Counter
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import difflib
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
!git clone https://github.com/ethelbeluzzi/projetomachado

Cloning into 'projetomachado'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 65 (delta 24), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (65/65), 7.21 MiB | 3.45 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [3]:
# Define o tamanho do contexto: 9 palavras de entrada, e a próxima palavra será o alvo (target).
context_size = 9

# TODO: Preparar o dataset (um lembrete para o programador para adicionar mais código futuramente)
"""TODO: Preparar o dataset"""

# Caminho do arquivo contendo o texto normalizado
DATA_PATH = os.path.join("projetomachado", "textonormalizado1000.txt")

# Abrir o arquivo de texto no modo de leitura ('r') e armazenar seu conteúdo em 'data_text'
with open(DATA_PATH, "r") as data_file:
    data_text = data_file.read()

# Divide o texto lido em uma lista de linhas (cada quebra de linha separa uma nova linha)
text_lines = data_text.split("\n")

In [4]:
# Função para normalizar o texto removendo elementos indesejados
def normalize_text(text):
    # Remove números romanos (com letras maiúsculas) que aparecem isolados
    text = re.sub(r'\b[IVXLCDM]+\b', '', text)
    # Remove números
    text = re.sub(r'\d+', '', text)
    # Remove URLs
    text = re.sub(r'http[s]?://\S+|www\.\S+|http:\S+', '', text)
    # Remove reticências (...)
    text = re.sub(r'\.\.\.', '', text)
    # Converte todo o texto para minúsculas
    text = text.lower()
    # Retorna o texto normalizado
    return text

# Função para filtrar as linhas com base no número mínimo de palavras
def filter_lines(lines, min_word_count=6):
    filtered_lines = []  # Lista para armazenar as linhas filtradas
    for line in lines:
        # Normaliza o texto de cada linha
        normalized_line = normalize_text(line)
        # Adiciona à lista apenas se a linha tiver palavras e número mínimo de palavras
        if normalized_line and len(normalized_line.split()) >= min_word_count:
            filtered_lines.append(normalized_line)
    return filtered_lines  # Retorna a lista de linhas filtradas

# Filtra e normaliza as linhas do texto
text_lines_normalized = filter_lines(text_lines)

In [5]:
# Exibe o segundo item (linha) da lista 'text_lines_normalized'
text_lines_normalized[1]

'morrese. quem não padece estas dores não as pode avaliar. o golpe foi profundo, e o'

In [6]:
# Junta todas as linhas normalizadas em uma única string, separando-as por espaços
full_data = ' '.join(text_lines_normalized)

# Retorna o comprimento (número de caracteres) do texto unificado
len(full_data)

# Exibe os primeiros 1000 caracteres do texto unificado
full_data[:1000]

'morrer? que idéia! deixate disso, estêvão. não se morre por tão pouco morrese. quem não padece estas dores não as pode avaliar. o golpe foi profundo, e o meu coração é pusilânime; por mais aborrecível que pareça a idéia da morte, pior, muito pior do que ela, é a de viver. ah! tu não sabes o que isto é?  e se em cada caso de namoro gorado morresse um homem, tinha já diminuído muito o gênero humano, e malthus perderia o latim. anda, sobe. estêvão meteu a mão nos cabelos com um gesto de angústia; luís alves sacudiu a cabeça e sorriu. achavamse os dois no corredor da casa de luís alves, à rua da constituição,  que então se chamava dos ciganos;  então, isto é, em , uma bagatela de vinte anos que lá vão, levando talvez consigo as ilusões do leitor, e deixandolhe em troca usurários! uma triste, crua e eram nove horas da noite; luís alves recolhiase para casa, justamente na ocasião em que estêvão o ia procurar; encontraramse à porta. ali mesmo lhe confiou estêvão tudo o que havia, e que o lei

## Criando um vocabulário

In [7]:
# Função para contar a ocorrência de palavras em uma lista de textos
def count_words(texts):
    word_counts = Counter()  # Inicializa um contador de palavras
    # Itera sobre cada texto e encontra as palavras ou símbolos, mantendo a pontuação
    for text in texts:
        word_counts.update(re.findall(r'\w+|\S', text.lower()))  # Converte para minúsculas e conta as palavras
    return word_counts

# Conta palavras no conjunto de textos normalizados
word_counts = count_words(text_lines_normalized)

# Define o tamanho do vocabulário (máximo de palavras) e número de tokens especiais
vocab_size = 5000  # Tamanho total do vocabulário
num_special_tokens = 4  # Tokens especiais como [PAD], [SOS], etc.
max_vocab_size = vocab_size - num_special_tokens  # Reduz o vocabulário para ajustar os tokens especiais

# Seleciona as palavras mais frequentes, de acordo com o limite estabelecido
most_frequent_words = [word for word, count in word_counts.most_common(max_vocab_size)]

# Cria o dicionário do vocabulário com as palavras mais frequentes e seus índices
vocab = {word: i + num_special_tokens for i, word in enumerate(most_frequent_words)}

# Adiciona tokens especiais ao vocabulário com índices predefinidos
vocab['[PAD]']  = 3  # Token para padding
vocab['[SOS]']  = 2  # Token para início de sentença
vocab['[EOS]']  = 1  # Token para fim de sentença
vocab['[UNK]']  = 0  # Token para palavras desconhecidas

# Exibe o tamanho do vocabulário criado
print(len(vocab))

# Exibe as 20 palavras mais frequentes
print("Most frequent words:", most_frequent_words[:20])

# Exibe as 20 palavras menos frequentes do vocabulário selecionado
print("Least frequent words:", most_frequent_words[-20:])

5000
Most frequent words: [',', '.', 'a', 'que', 'de', 'e', 'o', 'não', ';', 'um', 'do', 'da', 'os', 'é', 'com', 'uma', 'se', 'em', 'para', 'mas']
Least frequent words: ['orquestra', 'comeu', 'mordia', 'reinado', 'senteime', 'mostravam', 'falaria', 'encontrado', 'perguntasse', 'referia', 'resume', 'navio', 'valiam', 'suave', 'eternos', 'comerciante', 'conservatório', 'lúcio', 'recordando', 'postas']


In [8]:

# Função para encontrar a palavra mais próxima do vocabulário com base na similaridade de string
def find_closest_word(unknown_word, vocab):
    closest_word = difflib.get_close_matches(unknown_word, vocab.keys(), n=1, cutoff=0.6)
    return closest_word[0] if closest_word else '<UNK>'

# Função para codificar uma sentença (transformá-la em uma sequência de índices do vocabulário)
def encode_sentence(sentence, vocab):
    # Converte a sentença em palavras (ou símbolos), e mapeia cada palavra ao seu índice no vocabulário
    # Se a palavra não estiver no vocabulário, retorna o índice 0 ([UNK])
    return [vocab.get(word, 0) for word in re.findall(r'\w+|\S', sentence.lower())]

# Função para decodificar uma sequência de índices em uma sentença de palavras
def decode_sentence(encoded_sentence, vocab):
    # Cria um dicionário inverso (index -> palavra) para poder mapear os índices de volta para palavras
    reverse_vocab = {index: word for word, index in vocab.items()}

    decoded_sentence = []
    for index in encoded_sentence:
        word = reverse_vocab.get(index, None)
        if word is None:
            # Se o índice não estiver no vocabulário, tenta encontrar a palavra mais próxima
            unknown_word = '<UNK>'
            closest_word = find_closest_word(unknown_word, vocab)
            decoded_sentence.append(closest_word)
        else:
            decoded_sentence.append(word)

    return ' '.join(decoded_sentence)

# Exemplo de codificação e decodificação
print('Encoder:')
encoded = encode_sentence(text_lines_normalized[15], vocab)
print(encoded)  # Exibe a versão codificada da 16ª linha de texto (index 15)
print(text_lines_normalized[15])  # Exibe o texto original da linha 16

print('Decoder:')
decoded = decode_sentence(encoded, vocab)
print(decoded)  # Exibe a versão decodificada da linha codificada com substituição


Encoder:
[18, 10, 65, 22, 7, 0, 4, 10, 65, 6, 0, 7, 248, 129, 532, 4, 69, 0, 246, 4, 7, 11]
com o outro para que subisse, o outro a teimar que queria ir morrer, tão tenazes ambos, que não
Decoder:
com o outro para que [UNK] , o outro a [UNK] que queria ir morrer , tão [UNK] ambos , que não


In [9]:
# Dividindo em conjuntos de treino e validação
train_data, val_data = train_test_split(
    text_lines_normalized,
    test_size=0.2,
    random_state=18)

# Não utilize o split val para nada a partir daqui, somente validar

## Classe do dataset

In [10]:
# Define o tamanho do contexto: 9 palavras de entrada, a próxima palavra é o alvo (target)
context_size = 9

# Definição dos tokens especiais a partir do vocabulário
SOS = vocab['[SOS]']
EOS = vocab['[EOS]']
PAD = vocab['[PAD]']
UNK = vocab['[UNK]']

# Criação do dataset personalizado para o texto de Machado
class MachadoDataset(Dataset):
    def __init__(self, text, context_size, vocab):
        self.text = text  # Texto de entrada (linhas normalizadas)
        self.context_size = context_size  # Tamanho do contexto (número de palavras de entrada)
        self.vocab = vocab  # Vocabulário (mapeamento de palavras para índices)
        self.input_target_pairs = self.prepare_dataset()  # Prepara as entradas e alvos

    # Função para preparar o dataset com pares de entrada e alvo
    def prepare_dataset(self):
        input_target_pairs = []

        # Itera sobre cada linha de texto
        for item in self.text:
            # Codifica a sentença e adiciona os tokens SOS e EOS
            tokens = [SOS] + encode_sentence(item, self.vocab) + [EOS]

            # Trunca os tokens para múltiplos do tamanho do contexto
            max_len = ((len(tokens) - 1) // self.context_size) * self.context_size
            tokens = tokens[:max_len]  # Limita o comprimento dos tokens
            tokens.extend([EOS])  # Adiciona o token EOS ao final

            # Desliza uma janela sobre os tokens para criar pares de entrada e alvo
            for i in range(len(tokens) - self.context_size):
                context = tokens[i:i + self.context_size]  # Contexto de entrada
                target = tokens[i + 1:i + self.context_size + 1]  # Alvo (próximas palavras)

                # Substitui tokens UNK por PAD para evitar palavras desconhecidas
                target = [PAD if token == UNK else token for token in target]

                # Adiciona o par de tensores de entrada e alvo à lista
                input_target_pairs.append((torch.tensor(context), torch.tensor(target)))

        return input_target_pairs

    # Retorna o tamanho do dataset (número de pares de entrada e alvo)
    def __len__(self):
        return len(self.input_target_pairs)

    # Retorna o par de entrada e alvo correspondente ao índice fornecido
    def __getitem__(self, idx):
        return self.input_target_pairs[idx]


In [11]:
# Criação dos datasets de treinamento e validação usando a classe MachadoDataset
train_data = MachadoDataset(train_data, context_size, vocab)
val_data = MachadoDataset(val_data, context_size, vocab)

In [12]:
# Define o tamanho do lote (batch) e cria os DataLoaders para treinamento e validação
batch_size = 1000  # Define o tamanho de cada lote de dados (aumentado para 1000)

# Cria o DataLoader para o conjunto de treinamento, com shuffle ativado para misturar os dados
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# Cria o DataLoader para o conjunto de validação, também com shuffle ativado
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

# Pega um lote de exemplos do DataLoader de treinamento
sample = next(iter(train_loader))

# Exibe a forma (shape) do primeiro elemento do lote (entrada do modelo)
sample[0].shape


torch.Size([1000, 9])

## Model

In [13]:
# Multi-Head Self-Attention Layer
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, model_dim, heads_count):
        super(MultiHeadSelfAttention, self).__init__()
        # Verifica se a dimensão do modelo pode ser dividida igualmente pelos "heads"
        assert model_dim % heads_count == 0

        # Define o número de cabeças (heads) e a profundidade de cada cabeça
        self.heads_count = heads_count
        self.depth = model_dim // heads_count

        # Camadas lineares para gerar queries, keys, e values
        self.value_layer = nn.Linear(model_dim, model_dim, bias=False)
        self.key_layer = nn.Linear(model_dim, model_dim, bias=False)
        self.query_layer = nn.Linear(model_dim, model_dim, bias=False)
        self.output_layer = nn.Linear(model_dim, model_dim, bias=False)

    # Função para reformatar a entrada para dividir em várias cabeças
    def reshape_heads(self, x):
        batch_size, seq_len, model_dim = x.size()
        x = x.view(batch_size, seq_len, self.heads_count, self.depth)
        return x.transpose(1, 2)

    # Função que calcula o produto escalar (dot-product) com escala para a atenção
    def scaled_dot_product(self, Q, K, V):
        score = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.depth)
        attention_weights = torch.softmax(score, dim=-1)
        output = torch.matmul(attention_weights, V)
        return output, attention_weights

    # Função para "juntar" as várias cabeças em uma única saída
    def merge_heads(self, x):
        batch_size, heads_count, seq_len, depth = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_len, heads_count * depth)

    # Função de passagem do modelo (forward pass)
    def forward(self, queries, keys, values):
        # Gera Q (queries), K (keys) e V (values)
        Q = self.query_layer(queries)
        K = self.key_layer(keys)
        V = self.value_layer(values)

        # Reorganiza para múltiplas cabeças
        Q = self.reshape_heads(Q)
        K = self.reshape_heads(K)
        V = self.reshape_heads(V)

        # Calcula a atenção e os pesos
        attn_output, attn_weights = self.scaled_dot_product(Q, K, V)
        merged = self.merge_heads(attn_output)
        final_output = self.output_layer(merged)

        return final_output, attn_weights

# Camada de Decoder do Transformer
class TransformerDecoderLayer(nn.Module):
    def __init__(self, model_dim, heads_count, hidden_dim, dropout_rate=0.1):
        super(TransformerDecoderLayer, self).__init__()
        self.self_attention = MultiHeadSelfAttention(model_dim, heads_count)

        # Normalização e camadas fully connected
        self.layer_norm1 = nn.LayerNorm(model_dim)
        self.layer_norm2 = nn.LayerNorm(model_dim)

        self.fc1 = nn.Linear(model_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, model_dim)

        self.dropout1 = nn.Dropout(dropout_rate)
        self.dropout2 = nn.Dropout(dropout_rate)

    # Função de passagem do modelo
    def forward(self, x):
        attn_output, _ = self.self_attention(x, x, x)  # Autoatenção
        x = x + self.dropout1(attn_output)  # Residual connection + dropout
        x = self.layer_norm1(x)  # Normalização

        ff_output = self.fc2(F.relu(self.fc1(x)))  # Rede feed-forward
        x = x + self.dropout2(ff_output)  # Residual connection + dropout
        x = self.layer_norm2(x)  # Normalização
        return x

# Codificação Posicional para adicionar informações de posição ao modelo
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    # Adiciona a codificação posicional aos embeddings
    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return x

# Modelo completo de linguagem baseado em Transformer
class LanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, hidden_dim, num_heads, num_layers, dropout):
        super(LanguageModel, self).__init__()

        # Camadas de embedding e codificação posicional
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.positional_encoding = PositionalEncoding(embedding_dim, context_size)

        # Múltiplas camadas de Transformer Decoder
        self.decoder_layers = nn.ModuleList([
            TransformerDecoderLayer(embedding_dim, num_heads, hidden_dim, dropout)
            for _ in range(num_layers)
        ])

        # Camadas fully connected para prever o vocabulário
        self.fc1 = nn.Linear(embedding_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, vocab_size)

    # Função de passagem do modelo
    def forward(self, inputs):
        o = self.embeddings(inputs)
        o = self.positional_encoding(o)

        # Passa o embedding pelas camadas do decoder
        for layer in self.decoder_layers:
            o = layer(o)

        # Rede fully connected para previsão final
        o = F.relu(self.fc1(o))
        o = F.relu(self.fc2(o))
        o = self.fc3(o)

        return o

# Inicialização do modelo
embedding_dim = 64  # Tamanho da representação das palavras (model_dim)
vocab_size = 5000
context_size = 9
hidden_dim = embedding_dim * context_size
num_layers = 2
num_heads = 4
dropout = 0.3

# Criação do modelo de linguagem
model = LanguageModel(vocab_size, embedding_dim, context_size, hidden_dim, num_heads, num_layers, dropout)


In [14]:
# Pega o primeiro lote de exemplos do DataLoader de treinamento
sample = next(iter(train_loader))

# Separa o lote em entradas e alvos (targets)
input = sample[0]  # Entradas do modelo (contexto)
target = sample[1]  # Alvos do modelo (palavras subsequentes)


In [15]:
# Passa o lote de entrada pelo modelo para obter a saída
output = model(input)

# Obtém o índice da palavra com a maior probabilidade para cada sequência de saída
predicted = output.argmax(dim=1)

target

tensor([[2301,   12,    9,  ...,  725,   25,    6],
        [  43,   10,   25,  ...,    6, 4326,    4],
        [ 229,    4,    7,  ...,    3,   10,  600],
        ...,
        [ 522,   27,  176,  ...,    3,    4,    1],
        [2290,   37,    3,  ..., 1412,    3,    1],
        [   4,    7,   10,  ...,  202,    7,    1]])

## Training

In [16]:
# Verifica se há uma GPU disponível e define o dispositivo de execução
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move o modelo para o dispositivo (GPU se disponível, ou CPU caso contrário)
model.to(device)


LanguageModel(
  (embeddings): Embedding(5000, 64)
  (positional_encoding): PositionalEncoding()
  (decoder_layers): ModuleList(
    (0-1): 2 x TransformerDecoderLayer(
      (self_attention): MultiHeadSelfAttention(
        (value_layer): Linear(in_features=64, out_features=64, bias=False)
        (key_layer): Linear(in_features=64, out_features=64, bias=False)
        (query_layer): Linear(in_features=64, out_features=64, bias=False)
        (output_layer): Linear(in_features=64, out_features=64, bias=False)
      )
      (layer_norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (layer_norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=64, out_features=576, bias=True)
      (fc2): Linear(in_features=576, out_features=64, bias=True)
      (dropout1): Dropout(p=0.3, inplace=False)
      (dropout2): Dropout(p=0.3, inplace=False)
    )
  )
  (fc1): Linear(in_features=64, out_features=576, bias=True)
  (fc2): Linear(in_features=576

In [17]:
# Definição de hiperparâmetros
epochs = 10  # Número de épocas (quantas vezes o modelo verá o conjunto de dados completo)
lr = 1e-4  # Taxa de aprendizado (learning rate)
criterion = nn.CrossEntropyLoss(ignore_index=PAD)  # Função de perda, ignorando tokens PAD
optimizer = optim.AdamW(model.parameters(), lr=lr)  # Otimizador AdamW, ajustando os parâmetros do modelo

# Inicializa a perda acumulada para o conjunto de validação
val_running_loss = 0.0

# Modo de avaliação (não há backpropagation)
model.eval()
with torch.no_grad():  # Desativa o cálculo de gradientes (economiza memória e acelera a avaliação)
    for inputs, labels in val_loader:  # Itera sobre o conjunto de validação
        # Move os dados (entradas e rótulos) para o dispositivo correto (GPU ou CPU)
        inputs, labels = inputs.to(device), labels.to(device)

        # Passa os inputs pelo modelo para obter as previsões
        outputs = model(inputs)

        # Ajusta a dimensão das saídas para calcular a perda
        outputs = outputs.view(-1, vocab_size)  # Redimensiona para (batch_size * context_size, vocab_size)
        labels = labels.view(-1)  # Redimensiona para (batch_size * context_size)

        # Calcula a perda comparando as saídas preditas com os rótulos verdadeiros
        loss = criterion(outputs, labels)

        # Acumula a perda ponderada pelo tamanho do lote
        val_running_loss += loss.item() * inputs.size(0)

# Calcula a perda média por exemplo no conjunto de validação
initial_val_epoch_loss = val_running_loss / len(val_loader.dataset)

# Calcula a perplexidade (exponeciação da perda média)
initial_val_perplexity = np.exp(initial_val_epoch_loss)

# Imprime a perda e a perplexidade inicial no conjunto de validação
print(f'Initial Validation Loss: {initial_val_epoch_loss:.4f}, Initial Validation Perplexity: {initial_val_perplexity:.2f}')


Initial Validation Loss: 8.5295, Initial Validation Perplexity: 5061.90


In [18]:
# Listas para armazenar o loss e a acurácia por época
train_loss_history = []
val_loss_history = []
train_accuracy_history = []
val_accuracy_history = []

# Loop de treinamento por época
for epoch in range(epochs):
    start_time = time.time()  # Marca o início da época
    model.train()  # Coloca o modelo em modo de treinamento
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    # Loop de treinamento para cada batch
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Mover dados para GPU/CPU
        outputs = model(inputs)  # Passa os inputs pelo modelo

        # Ajustar dimensões para a função de perda
        outputs = outputs.view(-1, vocab_size)  # Formato para calcular a perda
        labels = labels.view(-1)

        # Calcular a perda
        loss = criterion(outputs, labels)

        # Backpropagation
        optimizer.zero_grad()  # Zera os gradientes acumulados
        loss.backward()  # Calcula os gradientes
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Prevenir gradientes explodindo
        optimizer.step()  # Atualiza os parâmetros

        running_loss += loss.item() * inputs.size(0)  # Acumula a perda para calcular a média depois

        # Calcular a acurácia
        _, predicted = torch.max(outputs, 1)  # Obter a predição com maior probabilidade
        correct_predictions += (predicted == labels).sum().item()  # Contar predições corretas
        total_predictions += labels.size(0)  # Contar o total de predições

    # Calcula a perda média e acurácia de treinamento por época
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = correct_predictions / total_predictions

    # Calcular a perplexidade (exponeciação da perda)
    perplexity = np.exp(epoch_loss)

    end_time = time.time()  # Tempo final da época
    epoch_duration = end_time - start_time  # Duração da época

    #### Validação
    model.eval()  # Coloca o modelo em modo de validação
    val_running_loss = 0.0
    val_correct_predictions = 0
    val_total_predictions = 0

    # Desabilitar gradientes durante a validação para economizar memória
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            # Ajustar dimensões para a função de perda
            outputs = outputs.view(-1, vocab_size)
            labels = labels.view(-1)

            # Calcular a perda
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * inputs.size(0)

            # Calcular a acurácia de validação
            _, predicted = torch.max(outputs, 1)
            val_correct_predictions += (predicted == labels).sum().item()
            val_total_predictions += labels.size(0)

    # Calcula a perda e acurácia média de validação por época
    val_epoch_loss = val_running_loss / len(val_loader.dataset)
    val_epoch_accuracy = val_correct_predictions / val_total_predictions

    # Calcular a perplexidade para o conjunto de validação
    val_perplexity = np.exp(val_epoch_loss)

    # Armazenar os valores de perda e acurácia para visualização posterior
    train_loss_history.append(epoch_loss)
    val_loss_history.append(val_epoch_loss)
    train_accuracy_history.append(epoch_accuracy)
    val_accuracy_history.append(val_epoch_accuracy)

    # Exibir os resultados da época atual
    print(f'Epoch [{epoch+1}/{epochs}], '
          f'Loss: {epoch_loss:.4f}, '
          f'Accuracy: {epoch_accuracy:.4f}, '
          f'Perplexity: {perplexity:.2f}, '
          f'Elapsed Time: {epoch_duration:.2f} sec, '
          f'Validation Loss: {val_epoch_loss:.4f}, '
          f'Validation Accuracy: {val_epoch_accuracy:.4f}, '
          f'Validation Perplexity: {val_perplexity:.2f}')


Epoch [1/10], Loss: 4.3508, Accuracy: 0.3156, Perplexity: 77.54, Elapsed Time: 58.13 sec, Validation Loss: 2.2009, Validation Accuracy: 0.5918, Validation Perplexity: 9.03
Epoch [2/10], Loss: 1.6952, Accuracy: 0.6522, Perplexity: 5.45, Elapsed Time: 60.03 sec, Validation Loss: 0.9654, Validation Accuracy: 0.7622, Validation Perplexity: 2.63
Epoch [3/10], Loss: 1.0215, Accuracy: 0.7486, Perplexity: 2.78, Elapsed Time: 62.26 sec, Validation Loss: 0.7267, Validation Accuracy: 0.7962, Validation Perplexity: 2.07
Epoch [4/10], Loss: 0.7971, Accuracy: 0.7813, Perplexity: 2.22, Elapsed Time: 63.52 sec, Validation Loss: 0.6277, Validation Accuracy: 0.8090, Validation Perplexity: 1.87
Epoch [5/10], Loss: 0.6872, Accuracy: 0.7966, Perplexity: 1.99, Elapsed Time: 63.74 sec, Validation Loss: 0.5787, Validation Accuracy: 0.8152, Validation Perplexity: 1.78
Epoch [6/10], Loss: 0.6261, Accuracy: 0.8050, Perplexity: 1.87, Elapsed Time: 63.65 sec, Validation Loss: 0.5508, Validation Accuracy: 0.8183, V

In [19]:
# Cria um subplot com dois gráficos lado a lado
fig = make_subplots(rows=1, cols=2, subplot_titles=("Perda ao longo das Épocas", "Acurácia ao longo das Épocas"))

# Adiciona o gráfico de Perda (Loss)
fig.add_trace(go.Scatter(x=list(range(len(train_loss_history))),
                         y=train_loss_history,
                         mode='lines',
                         name='Perda no Treinamento'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=list(range(len(val_loss_history))),
                         y=val_loss_history,
                         mode='lines',
                         name='Perda na Validação'),
              row=1, col=1)

# Adiciona o gráfico de Acurácia
fig.add_trace(go.Scatter(x=list(range(len(train_accuracy_history))),
                         y=train_accuracy_history,
                         mode='lines',
                         name='Acurácia no Treinamento'),
              row=1, col=2)

fig.add_trace(go.Scatter(x=list(range(len(val_accuracy_history))),
                         y=val_accuracy_history,
                         mode='lines',
                         name='Acurácia na Validação'),
              row=1, col=2)

# Atualiza os títulos e os eixos dos gráficos individualmente
fig.update_xaxes(title_text="Épocas", row=1, col=1)
fig.update_xaxes(title_text="Épocas", row=1, col=2)

fig.update_yaxes(title_text="Perda", row=1, col=1)
fig.update_yaxes(title_text="Acurácia", row=1, col=2)

# Atualiza o layout geral do gráfico
fig.update_layout(height=600, width=1200,
                  title_text="Perda e Acurácia ao longo das Épocas")

# Exibe o gráfico
fig.show()


## Exemplo de uso

In [20]:
def generate_text(model, vocab, text, max_length, context):
    """Gere texto até atingir max_length usando o modelo e o vocabulário fornecidos."""
    model.eval()
    encoded_text = encode_sentence(text, vocab)
    encoded_text_tensor = torch.tensor(encoded_text, device=device).unsqueeze(0)
    generated_text = list(encoded_text)

    while len(generated_text) < max_length:
        # Obtenha o contexto para a previsão
        if len(generated_text) >= context:
            context_input = generated_text[-context:]
        else:
            context_input = [PAD] * (context - len(generated_text)) + generated_text
        context_input_tensor = torch.tensor(context_input, device=device).unsqueeze(0)

        # Gere a previsão para o próximo token
        with torch.no_grad():
            logits = model(context_input_tensor)  # Saída do modelo: [1, context, vocab_size]
            logits = logits[:, -1, :]  # Pegue a previsão para o último token do contexto
            probabilities = F.softmax(logits, dim=-1)  # Calcule as probabilidades
            next_token = torch.multinomial(probabilities[0], num_samples=1).item()

        # Se atingir o token EOS, pare a geração
        if next_token == EOS:
            break
        generated_text.append(next_token)

    # Decodifique o texto gerado
    generated_text_decoded = decode_sentence(generated_text, vocab)
    return generated_text_decoded

# Exemplos de uso
t1 = "uma triste, crua e eram nove horas da noite"
context = 9
max_length = 50

text2 = "luís alves sacudiu a cabeça e sorriu"
context = 9
max_length = 50

text3 = "o golpe foi profundo"
context = 9
max_length = 50

text4 = "uma bagatela de vinte anos"
context = 9
max_length = 50

text5 = "use morre por tão pouco morrese"
context = 9
max_length = 50

# Gerar e imprimir 3 frases de forma randômica do t1
print("Texto 1")
for i in range(3):
    generated_text = generate_text(model, vocab, t1, max_length, context)
    print(f"Frase {i+1}: {generated_text}\n")

# Gerar e imprimir 3 frases de forma randômica do t2
print("Texto 2")
for i in range(3):
    generated_text = generate_text(model, vocab, text2, max_length, context)
    print(f"Frase {i+1}: {generated_text}\n")

# Gerar e imprimir 3 frases de forma randômica do t3
print("Texto 3")
for i in range(3):
    generated_text = generate_text(model, vocab, text3, max_length, context)
    print(f"Frase {i+1}: {generated_text}\n")

# Gerar e imprimir 3 frases de forma randômica do t4
print("Texto 4")
for i in range(3):
    generated_text = generate_text(model, vocab, text4, max_length, context)
    print(f"Frase {i+1}: {generated_text}\n")

    # Gerar e imprimir 3 frases de forma randômica do t5
print("Texto 5")
for i in range(3):
    generated_text = generate_text(model, vocab, text5, max_length, context)
    print(f"Frase {i+1}: {generated_text}\n")

Texto 1
Frase 1: uma triste , [UNK] e eram nove horas da noite , não tinha de causa da terra , onde mal , e

Frase 2: uma triste , [UNK] e eram nove horas da noite , a

Frase 3: uma triste , [UNK] e eram nove horas da noite filho e que ela a

Texto 2
Frase 1: luís alves sacudiu a cabeça e sorriu é a partida , e o pai , ou

Frase 2: luís alves sacudiu a cabeça e sorriu faria , como a cabeça , a renan . entrou na vocação respondeu alma

Frase 3: luís alves sacudiu a cabeça e sorriu , ou risco e muito uma idéia o diabo . não admira , nem

Texto 3
Frase 1: o golpe foi profundo ; ouvi nós o acaso a infeliz já na arte . ao pé . o cunhado , e para

Frase 2: o golpe foi profundo . mas há

Frase 3: o golpe foi profundo na sege . ele exige ciúmes . creio tudo

Texto 4
Frase 1: uma [UNK] de vinte anos , ergueuse cinco em si para crer maria ver um .

Frase 2: uma [UNK] de vinte anos

Frase 3: uma [UNK] de vinte anos , empregado não se sucedeu , de deixar regras . eu fazia que vinha da morte ; me d

## Refinamento com LoRA

In [21]:
# Definição da camada LoRA
class LoRALayer(nn.Module):
    """
    Define uma camada LoRA que consiste em duas matrizes A e B.

    Args:
        in_dim: Dimensão de entrada (número de características ou características de entrada)
        out_dim: Dimensão de saída
        rank: Dimensão de redução (ou posto) da camada LoRA, que reduz a dimensionalidade
        alpha: Fator de escala para ajustar a contribuição da LoRA ao modelo principal
    """
    def __init__(self, in_dim, out_dim, rank, alpha=1):
        super().__init__()

        # Define o desvio padrão usado na inicialização da matriz A.
        # O desvio padrão é calculado como o inverso da raiz quadrada do rank.
        std_dev = 1 / torch.sqrt(torch.tensor(rank).float())

        # Inicializa a matriz A com valores aleatórios normalizados pelo desvio padrão calculado.
        # A tem dimensão (in_dim, rank), o que significa que estamos reduzindo a dimensionalidade da entrada.
        self.A = nn.Parameter(torch.randn(in_dim, rank) * std_dev)

        # Inicializa a matriz B com zeros. B tem dimensão (rank, out_dim),
        # que expande de volta para a dimensão de saída desejada.
        self.B = nn.Parameter(torch.zeros(rank, out_dim))

        # O fator de escala alpha ajusta a magnitude da transformação LoRA.
        self.alpha = alpha

    def forward(self, x):
        # No método forward, a entrada x é multiplicada pela matriz A (redução da dimensionalidade),
        # seguida da multiplicação pela matriz B (expansão da dimensionalidade de volta ao tamanho original).
        # O resultado é escalado por alpha e retornado como a saída da camada LoRA.
        return self.alpha * (x @ self.A @ self.B)


# Definição de uma camada linear combinada com LoRA
class LinearWithLoRA(nn.Module):
    """
    Combina uma camada linear tradicional com uma camada LoRA.

    Args:
        linear: Uma camada linear do PyTorch, já existente
        rank: Dimensão de redução (ou posto) da LoRA
        alpha: Fator de escala para ajustar a contribuição da LoRA
    """
    def __init__(self, linear, rank, alpha=1):
        super().__init__()

        # Armazena a camada linear que será usada em conjunto com LoRA.
        self.linear = linear

        # Cria a camada LoRA correspondente com a mesma dimensão de entrada e saída da camada linear.
        # A LoRA será aplicada paralelamente à camada linear para ajustar o aprendizado.
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        # O método forward aplica a camada linear tradicional e a camada LoRA separadamente.
        # A saída final é a soma das duas saídas, combinando a transformação linear padrão com o ajuste LoRA.
        return self.linear(x) + self.lora(x)


# Definição de uma camada de embedding combinada com LoRA
class EmbeddingWithLoRA(nn.Module):
    """
    Combina uma camada de embeddings com uma camada LoRA.

    Args:
        embed: Uma camada de embeddings do PyTorch
        rank: Dimensão de redução (ou posto) da LoRA
        alpha: Fator de escala para ajustar a contribuição da LoRA
    """
    def __init__(self, embed, rank, alpha=1):
        super().__init__()

        # Armazena a camada de embedding original.
        self.embedding = embed

        # Cria uma camada LoRA que atua na dimensão dos embeddings.
        # Tanto a dimensão de entrada quanto a de saída da LoRA são iguais à dimensão dos embeddings.
        self.lora = LoRALayer(
            embed.embedding_dim, embed.embedding_dim, rank, alpha
        )

    def forward(self, x):
        # O método forward primeiro obtém os embeddings correspondentes para as entradas x.
        embed = self.embedding(x)

        # Em seguida, aplica a transformação LoRA sobre os embeddings.
        lora_output = self.lora(embed)

        # A saída final é a soma dos embeddings originais com o ajuste produzido pela LoRA.
        return embed + lora_output


In [35]:
# Inspecionar a estrutura do modelo
print(model)

LanguageModel(
  (embeddings): Embedding(5000, 64)
  (positional_encoding): PositionalEncoding()
  (decoder_layers): ModuleList(
    (0-1): 2 x TransformerDecoderLayer(
      (self_attention): MultiHeadSelfAttention(
        (value_layer): Linear(in_features=64, out_features=64, bias=False)
        (key_layer): Linear(in_features=64, out_features=64, bias=False)
        (query_layer): Linear(in_features=64, out_features=64, bias=False)
        (output_layer): Linear(in_features=64, out_features=64, bias=False)
      )
      (layer_norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (layer_norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=64, out_features=576, bias=True)
      (fc2): Linear(in_features=576, out_features=64, bias=True)
      (dropout1): Dropout(p=0.3, inplace=False)
      (dropout2): Dropout(p=0.3, inplace=False)
    )
  )
  (fc1): Linear(in_features=64, out_features=576, bias=True)
  (fc2): Linear(in_features=576

In [34]:
# Aplicar LoRA ao modelo
import copy

# Definição dos hiperparâmetros do LoRA
lora_rank = 16
lora_alpha = 2

model_lora = copy.deepcopy(model)

# Congelar todos os parâmetros do modelo
for param in model_lora.parameters():
    param.requires_grad = False

# Aplicar modificações LoRA às embeddings
model_lora.embeddings = EmbeddingWithLoRA(model_lora.embeddings, rank=lora_rank, alpha=lora_alpha)

# Aplicar LoRA às camadas do transformador do modelo usando 'query_layer', 'key_layer', 'value_layer' e 'output_layer'
for layer in model_lora.decoder_layers:
    layer.self_attention.query_layer = LinearWithLoRA(layer.self_attention.query_layer, rank=lora_rank, alpha=lora_alpha)
    layer.self_attention.key_layer = LinearWithLoRA(layer.self_attention.key_layer, rank=lora_rank, alpha=lora_alpha)
    layer.self_attention.value_layer = LinearWithLoRA(layer.self_attention.value_layer, rank=lora_rank, alpha=lora_alpha)
    layer.self_attention.output_layer = LinearWithLoRA(layer.self_attention.output_layer, rank=lora_rank, alpha=lora_alpha)

    # Aplicar LoRA às camadas feedforward
    layer.fc1 = LinearWithLoRA(layer.fc1, rank=lora_rank, alpha=lora_alpha)
    layer.fc2 = LinearWithLoRA(layer.fc2, rank=lora_rank, alpha=lora_alpha)

model_lora.fc1 = LinearWithLoRA(model_lora.fc1, rank=lora_rank, alpha=lora_alpha)
model_lora.fc2 = LinearWithLoRA(model_lora.fc2, rank=lora_rank, alpha=lora_alpha)

# Inspecionar a estrutura do modelo
print(model_lora)


LanguageModel(
  (embeddings): EmbeddingWithLoRA(
    (embedding): Embedding(5000, 64)
    (lora): LoRALayer()
  )
  (positional_encoding): PositionalEncoding()
  (decoder_layers): ModuleList(
    (0-1): 2 x TransformerDecoderLayer(
      (self_attention): MultiHeadSelfAttention(
        (value_layer): LinearWithLoRA(
          (linear): Linear(in_features=64, out_features=64, bias=False)
          (lora): LoRALayer()
        )
        (key_layer): LinearWithLoRA(
          (linear): Linear(in_features=64, out_features=64, bias=False)
          (lora): LoRALayer()
        )
        (query_layer): LinearWithLoRA(
          (linear): Linear(in_features=64, out_features=64, bias=False)
          (lora): LoRALayer()
        )
        (output_layer): LinearWithLoRA(
          (linear): Linear(in_features=64, out_features=64, bias=False)
          (lora): LoRALayer()
        )
      )
      (layer_norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (layer_norm2): LayerNorm((64

In [24]:
# Função para calcular a acurácia
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct_predictions = (predicted == labels).sum().item()
    return correct_predictions / labels.size(0)

# Função para calcular o loss e a acurácia inicial
def initial_evaluation(model, dataloader, criterion, device, vocab_size):
    model.eval()  # Coloca o modelo em modo de avaliação
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():  # Desabilita o cálculo de gradientes
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Obter as saídas do modelo LoRA
            outputs = model(inputs)

            # Ajustar dimensões para a função de perda
            outputs = outputs.view(-1, vocab_size)
            labels = labels.view(-1)

            # Calcular o loss
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            # Calcular a acurácia
            correct_predictions += (torch.argmax(outputs, dim=1) == labels).sum().item()
            total_predictions += labels.size(0)

    # Perda e acurácia médias
    avg_loss = running_loss / len(dataloader.dataset)
    avg_accuracy = correct_predictions / total_predictions
    perplexity = np.exp(avg_loss)

    return avg_loss, avg_accuracy, perplexity

# Avaliação inicial no conjunto de treinamento e validação
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_lora.to(device)  # Envia o modelo LoRA para o dispositivo

# Avaliar no conjunto de treinamento
train_loss, train_accuracy, train_perplexity = initial_evaluation(model_lora, train_loader, criterion, device, vocab_size)
print(f'Initial Training Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, Perplexity: {train_perplexity:.2f}')

# Avaliar no conjunto de validação
val_loss, val_accuracy, val_perplexity = initial_evaluation(model_lora, val_loader, criterion, device, vocab_size)
print(f'Initial Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Perplexity: {val_perplexity:.2f}')


Initial Training Loss: 0.4989, Accuracy: 0.8224, Perplexity: 1.65
Initial Validation Loss: 0.5062, Accuracy: 0.8231, Perplexity: 1.66


In [26]:
# Função de treinamento e avaliação
def train(model, train_loader, val_loader, criterion, optimizer, epochs, device):
    # Listas para armazenar histórico de loss e acurácia
    train_loss_history = []
    val_loss_history = []
    train_accuracy_history = []
    val_accuracy_history = []

    for epoch in range(epochs):
        # Treinamento
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zerar gradientes acumulados
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            outputs = outputs.view(-1, vocab_size)
            labels = labels.view(-1)

            # Cálculo do loss
            loss = criterion(outputs, labels)
            loss.backward()  # Backpropagation
            optimizer.step()  # Atualização dos pesos

            # Acumular loss e calcular acurácia
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

        # Loss e acurácia médios para o treinamento
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = correct_predictions / total_predictions
        train_loss_history.append(epoch_loss)
        train_accuracy_history.append(epoch_accuracy)

        # Validação
        model.eval()
        val_running_loss = 0.0
        val_correct_predictions = 0
        val_total_predictions = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                outputs = outputs.view(-1, vocab_size)
                labels = labels.view(-1)

                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct_predictions += (predicted == labels).sum().item()
                val_total_predictions += labels.size(0)

        # Loss e acurácia médias para a validação
        val_epoch_loss = val_running_loss / len(val_loader.dataset)
        val_epoch_accuracy = val_correct_predictions / val_total_predictions
        val_loss_history.append(val_epoch_loss)
        val_accuracy_history.append(val_epoch_accuracy)

        # Exibir resultados da época
        print(f'Epoch [{epoch+1}/{epochs}], '
              f'Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}, '
              f'Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_accuracy:.4f}')

    # Retorna o histórico de treinamento para análise posterior
    return {
        'train_loss': train_loss_history,
        'val_loss': val_loss_history,
        'train_accuracy': train_accuracy_history,
        'val_accuracy': val_accuracy_history
    }

# Definir os parâmetros do modelo e critério de perda
epochs = 10
lr = 1e-4
PAD_TOKEN = vocab['[PAD]']  # Corrigir a chamada para usar o vocabulário
criterion = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN)

# Definir o otimizador AdamW para o modelo LoRA
optimizer = torch.optim.AdamW(model_lora.parameters(), lr=lr)

# Treinar o modelo LoRA
history = train(model_lora, train_loader, val_loader, criterion, optimizer, epochs, device)


Epoch [1/10], Train Loss: 0.5221, Train Accuracy: 0.8186, Validation Loss: 0.5027, Validation Accuracy: 0.8235
Epoch [2/10], Train Loss: 0.5165, Train Accuracy: 0.8195, Validation Loss: 0.5005, Validation Accuracy: 0.8237
Epoch [3/10], Train Loss: 0.5126, Train Accuracy: 0.8200, Validation Loss: 0.4990, Validation Accuracy: 0.8238
Epoch [4/10], Train Loss: 0.5100, Train Accuracy: 0.8204, Validation Loss: 0.4975, Validation Accuracy: 0.8240
Epoch [5/10], Train Loss: 0.5076, Train Accuracy: 0.8207, Validation Loss: 0.4966, Validation Accuracy: 0.8241
Epoch [6/10], Train Loss: 0.5060, Train Accuracy: 0.8209, Validation Loss: 0.4958, Validation Accuracy: 0.8241
Epoch [7/10], Train Loss: 0.5049, Train Accuracy: 0.8210, Validation Loss: 0.4953, Validation Accuracy: 0.8242
Epoch [8/10], Train Loss: 0.5038, Train Accuracy: 0.8211, Validation Loss: 0.4952, Validation Accuracy: 0.8242
Epoch [9/10], Train Loss: 0.5028, Train Accuracy: 0.8212, Validation Loss: 0.4944, Validation Accuracy: 0.8242
E

In [27]:
# Cria um subplot com dois gráficos lado a lado
fig = make_subplots(rows=1, cols=2, subplot_titles=("Perda ao longo das Épocas", "Acurácia ao longo das Épocas"))

# Adiciona o gráfico de Perda (Loss)
fig.add_trace(go.Scatter(x=list(range(len(train_loss_history))),
                         y=train_loss_history,
                         mode='lines',
                         name='Perda no Treinamento'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=list(range(len(val_loss_history))),
                         y=val_loss_history,
                         mode='lines',
                         name='Perda na Validação'),
              row=1, col=1)

# Adiciona o gráfico de Acurácia
fig.add_trace(go.Scatter(x=list(range(len(train_accuracy_history))),
                         y=train_accuracy_history,
                         mode='lines',
                         name='Acurácia no Treinamento'),
              row=1, col=2)

fig.add_trace(go.Scatter(x=list(range(len(val_accuracy_history))),
                         y=val_accuracy_history,
                         mode='lines',
                         name='Acurácia na Validação'),
              row=1, col=2)

# Atualiza os títulos e os eixos dos gráficos individualmente
fig.update_xaxes(title_text="Épocas", row=1, col=1)
fig.update_xaxes(title_text="Épocas", row=1, col=2)

fig.update_yaxes(title_text="Perda", row=1, col=1)
fig.update_yaxes(title_text="Acurácia", row=1, col=2)

# Atualiza o layout geral do gráfico
fig.update_layout(height=600, width=1200, title_text="Perda e Acurácia ao longo das Épocas")

# Exibe o gráfico
fig.show()


In [33]:
def generate_text(model, vocab, text, max_length, context):
    """Gere texto até atingir max_length usando o modelo e o vocabulário fornecidos."""
    model.eval()
    encoded_text = encode_sentence(text, vocab)
    encoded_text_tensor = torch.tensor(encoded_text, device=device).unsqueeze(0)
    generated_text = list(encoded_text)

    while len(generated_text) < max_length:
        # Obtenha o contexto para a previsão
        if len(generated_text) >= context:
            context_input = generated_text[-context:]
        else:
            context_input = [PAD] * (context - len(generated_text)) + generated_text
        context_input_tensor = torch.tensor(context_input, device=device).unsqueeze(0)

        # Gere a previsão para o próximo token
        with torch.no_grad():
            logits = model(context_input_tensor)  # Saída do modelo: [1, context, vocab_size]
            logits = logits[:, -1, :]  # Pegue a previsão para o último token do contexto
            probabilities = F.softmax(logits, dim=-1)  # Calcule as probabilidades
            next_token = torch.multinomial(probabilities[0], num_samples=1).item()

        # Se atingir o token EOS, pare a geração
        if next_token == EOS:
            break
        generated_text.append(next_token)

    # Decodifique o texto gerado
    generated_text_decoded = decode_sentence(generated_text, vocab)
    return generated_text_decoded

# Exemplos de uso
t1 = "uma triste, crua e eram nove horas da noite"
context = 9
max_length = 50

text2 = "luís alves sacudiu a cabeça e sorriu"
context = 9
max_length = 50

text3 = "o golpe foi profundo"
context = 9
max_length = 50

text4 = "uma bagatela de vinte anos"
context = 9
max_length = 50

text5 = "use morre por tão pouco morrese"
context = 9
max_length = 50

# Gerar e imprimir 3 frases de forma randômica do t1
print("Texto 1")
for i in range(3):
    generated_text = generate_text(model, vocab, t1, max_length, context)
    print(f"Frase {i+1}: {generated_text}\n")

# Gerar e imprimir 3 frases de forma randômica do t2
print("Texto 2")
for i in range(3):
    generated_text = generate_text(model, vocab, text2, max_length, context)
    print(f"Frase {i+1}: {generated_text}\n")

# Gerar e imprimir 3 frases de forma randômica do t3
print("Texto 3")
for i in range(3):
    generated_text = generate_text(model, vocab, text3, max_length, context)
    print(f"Frase {i+1}: {generated_text}\n")

# Gerar e imprimir 3 frases de forma randômica do t4
print("Texto 4")
for i in range(3):
    generated_text = generate_text(model, vocab, text4, max_length, context)
    print(f"Frase {i+1}: {generated_text}\n")

    # Gerar e imprimir 3 frases de forma randômica do t5
print("Texto 5")
for i in range(3):
    generated_text = generate_text(model, vocab, text5, max_length, context)
    print(f"Frase {i+1}: {generated_text}\n")

Texto 1
Frase 1: uma triste , [UNK] e eram nove horas da noite .

Frase 2: uma triste , [UNK] e eram nove horas da noite , dizialhe me vir das três vezes . ora levantou me estava com

Frase 3: uma triste , [UNK] e eram nove horas da noite mais , era que eu fosse . a s bem

Texto 2
Frase 1: luís alves sacudiu a cabeça e sorriu

Frase 2: luís alves sacudiu a cabeça e sorriu . era a poesia , na mão fui dama sobre é ainda a cansado do mesmo à hora por causa de boa de a instituição , que vira vinha por mim então daquele

Frase 3: luís alves sacudiu a cabeça e sorriu ao filho que a última indiferença

Texto 3
Frase 1: o golpe foi profundo ,

Frase 2: o golpe foi profundo . quando o resto , dos

Frase 3: o golpe foi profundo . às dois degraus com ela não levantava nomes tarde de

Texto 4
Frase 1: uma [UNK] de vinte anos , tal

Frase 2: uma [UNK] de vinte anos cheia , a consulta e ele será eu me houve fora .

Frase 3: uma [UNK] de vinte anos ;

Texto 5
Frase 1: [UNK] morre por tão pouco [UNK] 